In [ ]:
%%bash
pip install --upgrade pip
pip install wheel
pip install setuptools
pip install --upgrade setuptools wheel
pip install --upgrade cmake pybind11
pip install nglview
pip install GromacsWrapper 


In [ ]:
import subprocess

# Define the installation command based on your system's package manager
# For Ubuntu/Debian-based systems:
install_command = "sudo apt-get install gromacs"

# For macOS using Homebrew:
# install_command = "brew install gromacs"

# Execute the installation command
try:
    subprocess.run(install_command, shell=True, check=True)
    print("GROMACS has been successfully installed.")
except subprocess.CalledProcessError:
    print("Error: Failed to install GROMACS.")
    
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np 
import gromacs 
import gromacs.formats
import nglview as nv


In [ ]:
!grep -v HOH maws_pfoa_aptamer.pdb > python_aptamer.pdb
!printf "1 1" | gmx pdb2gmx -f python_aptamer.pdb -o python_aptamer_processed.gro -ignh 
!gmx editconf -f python_aptamer_processed.gro -o python_newbox.gro -c -d 0.5 -bt cubic
!gmx solvate -cp python_newbox.gro -cs spc216.gro -o python_solv.gro -p topol.top
!gmx grompp -f ions.mdp -c python_solv.gro -p topol.top -o ions.tpr
!printf "SOL" | gmx genion -s ions.tpr -o python_solv_ions.gro -p topol.top -pname NA -nname CL -neutral
!gmx grompp -f minim.mdp -c python_solv_ions.gro -p topol.top -o em.tpr
!gmx mdrun -v -deffnm em
!printf "10 0"| gmx energy -f em.edr -o potential.xvg
!gmx grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr
!gmx mdrun -deffnm nvt
!printf "16 0" | gmx energy -f nvt.edr -o temperature.xvg
!gmx grompp -f npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr
!gmx mdrun -deffnm npt
!printf "18 0" | gmx energy -f npt.edr -o pressure.xvg
!gmx energy -f npt.edr -o density.xvg
!gmx grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md_python.tpr
!gmx mdrun -deffnm md_python

In [ ]:
!printf "1 0" | gmx trjconv -s md_python.tpr -f md_python.xtc -o md_python_noPBC.xtc -pbc mol -center
!printf "4 0" | gmx rms -s md_0_1.tpr -f md_python_noPBC.xtc -o rmsd.xvg -tu ns
!gmx rms -s em.tpr -f md_python_noPBC.xtc -o rmsd_xtal.xvg -tu ns
!printf "1 0" | gmx gyrate -s md_python.tpr -f md_python_noPBC.xtc -o gyrate.xvg
